In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 100_Unique_QA_Dataset.csv to 100_Unique_QA_Dataset.csv


In [3]:
df = pd.read_csv("100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [4]:
df.shape

(90, 2)

# Tokenization

In [5]:
# tokenize

def tokenize(text):
  text = text.lower()
  #text = text.replace("\n", " ")
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [6]:
# vocabulary

voc = {'unk' : 0}

def vocab(row):
  ques_token = tokenize(row['question'])
  ans_token = tokenize(row['answer'])

  merged_token = ques_token + ans_token

  for i in merged_token:
    if i not in voc:
      voc[i] = len(voc)

In [7]:
df.apply(vocab, axis=1)


,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [8]:
len(voc)

324

# Convert Words to vector form

In [9]:
# convert words to numerical indices

def word_to_vec(text, voc):

  indexed_text = []

  for i in tokenize(text):
    if i in voc:
      indexed_text.append(voc[i])
    else:
      indexed_text.append(voc['unk'])

  return indexed_text

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class qadataset(Dataset):

  def __init__(self, df, voc):
    self.df = df
    self.voc = voc

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    num_ques = word_to_vec(self.df.iloc[idx]['question'], self.voc)
    num_ans = word_to_vec(self.df.iloc[idx]['answer'], self.voc)

    return torch.tensor(num_ques), torch.tensor(num_ans)

In [12]:
dataset = qadataset(df, voc)

In [13]:
dataset[9]

(tensor([ 1,  2,  3, 50, 51, 19,  3, 45]), tensor([52]))

In [14]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [15]:
for ques, ans in dataloader:
  print(ques, ans)

tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([[199]])
tensor([[ 10,  75, 208]]) tensor([[209]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([[246]])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([[113]])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([[52]])
tensor([[10, 75, 76]]) tensor([[77]])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([[65]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([[179]])
tensor([[ 10,  96,   3, 104, 239]]) tensor([[240]])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([[215]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
te

# Model Creation

In [16]:
import torch.nn as nn

class simplernn(nn.Module):

  def __init__(self, voc_size):

    super().__init__()

    self.embedding = nn.Embedding(voc_size, embedding_dim = 50)    # every vector is converted into 50 dim vector
    self.rnn = nn.RNN(50, 64, batch_first = True)
    self.out = nn.Linear(64, voc_size)

  def forward(self, question):
    embedded_ques = self.embedding(question)
    hidden, final = self.rnn(embedded_ques)
    output = self.out(final.squeeze(0))

    return output

**nn.RNN** returns two outputs:

*   the output for all time steps (hidden states)
*   the final hidden state

As **nn.Sequential** expects each module to return a single tensor that is passed as input to the next layer, it **cannot be used** with **nn.RNN**, which returns a tuple instead of one output

In [17]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [18]:
learning_rate = 0.01
epochs = 20

In [19]:
model = simplernn(len(voc))

loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [20]:
for i in range(epochs):
  total_loss = 0

  for ques, ans in dataloader:

    opt.zero_grad()

    # forward pass
    pred = model(ques)

    # loss
    l = loss(pred, ans[0])

    # gradients
    l.backward()

    # update
    opt.step()

    total_loss += l.item()

  print(f"loss in epoch {i+1} = {total_loss}")

loss in epoch 1 = 540.0800495147705
loss in epoch 2 = 316.83334094285965
loss in epoch 3 = 145.7109732925892
loss in epoch 4 = 56.09399968292564
loss in epoch 5 = 27.492788252886385
loss in epoch 6 = 19.716688762418926
loss in epoch 7 = 33.89364823466167
loss in epoch 8 = 21.681059037335217
loss in epoch 9 = 12.694542235694826
loss in epoch 10 = 10.19531037681736
loss in epoch 11 = 14.143053100677207
loss in epoch 12 = 8.461478506214917
loss in epoch 13 = 4.316679506096989
loss in epoch 14 = 11.240971872815862
loss in epoch 15 = 6.107769100693986
loss in epoch 16 = 5.470225425204262
loss in epoch 17 = 11.086247478378937
loss in epoch 18 = 5.842241180827841
loss in epoch 19 = 5.545975252520293
loss in epoch 20 = 10.916619882220402


In [23]:
def predict(model, ques, threshold = 0.5):

  # convert question to numbers
  num_quest = word_to_vec(ques, voc)

  # tensor
  ques_tensor = torch.tensor(num_quest).unsqueeze(0)

  # send to model
  out = model(ques_tensor)        # these outputs are logits

  # convert logits to probs
  prob = torch.nn.functional.softmax(out, dim = 1)

  # find index of max prob
  value, idx = torch.max(prob, dim = 1)

  if value < threshold:
    print("I don't know")

  print(list(voc.keys())[idx])

In [24]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [25]:
predict(model, "What is campusx")

I don't know
7


In [26]:
predict(model, "What is my name")

I don't know
299,792,458m/s
